# Entrenamiento de una red YOLO para el dataset de trashnet
### modelo usado: 

In [1]:
from ultralytics import YOLO
from pathlib import Path
import os
import sys

In [2]:
#definicion de variables para el entrenamiento

MODEL_PATH = "./yolo/"
epochs = 100
imgsz = 416
batch_size = 16
optimizer = "SGD"
random_seed = 42

data_yaml = "./data/trashnet-anotado/data.yaml"
model_name = "yolo11n.pt"
model_path = os.path.join(MODEL_PATH, model_name)

print(model_path)

./yolo/yolo11n.pt


In [ ]:
#carga del modelo y entrenamiento

In [5]:
import torch
import sys

print (sys.executable)


print(torch.__version__)
print(torch.version.cuda)


if torch.cuda.is_available():
    print("CUDA is available.")
    print(f"Number of GPUs: {torch.cuda.device_count()}")
    print(f"Current GPU name: {torch.cuda.get_device_name(0)}")
else:
    print("CUDA is not available. PyTorch will run on CPU.")

/home/martin/Documents/CEIA/CEIA-vpc2/venv/bin/python
2.10.0.dev20250927+cu130
13.0
CUDA is not available. PyTorch will run on CPU.


In [4]:
model = YOLO(model_path)
results = model.train(
    data=data_yaml,
    epochs=epochs,
    imgsz=imgsz,
    batch=batch_size,
    name=model_name,
    device=0,
    workers=4,
    augment=True,
    patience=10,
    optimizer=optimizer,
    seed=random_seed
)

Ultralytics 8.3.203 🚀 Python-3.12.3 torch-2.10.0.dev20250927+cu130 


ValueError: Invalid CUDA 'device=0' requested. Use 'device=cpu' or pass valid CUDA device(s) if available, i.e. 'device=0' or 'device=0,1,2,3' for Multi-GPU.

torch.cuda.is_available(): False
torch.cuda.device_count(): 1
os.environ['CUDA_VISIBLE_DEVICES']: None
